In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import IPython
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2 as sklearn_chi2
from sklearn.preprocessing.imputation import Imputer

In [3]:
df = pd.read_csv(r"C:\Users\deepc\Desktop\Data Work\TestData.csv")
df1 = pd.read_csv(r"C:\Users\deepc\Desktop\Data Work\static.csv")
df1 = df1.drop(["Unnamed: 0"], axis = 1)

In [4]:
data = df.merge(df1,on='matchid',how='inner')

In [5]:
#Toss winner team = home team

In [6]:
data.head()

,matchid,Players,BattingOrder,inning_bat,Bat_Bowler,Bat_Dismissal,Bat_Runs,4s,6s,Ball Faced,...,player_of_match,reserve_umpire,tv_umpire,match_referee,outcome,method,neutralvenue,winner,winner_runs,winner_wickets
0,1000851,SC Cook,1,1,MA Starc,caught,0.0,0.0,0.0,4.0,...,K Rabada,SJ Nogajski,RA Kettleborough,AJ Pycroft,NaN,NaN,NaN,South Africa,177.0,NaN
1,1000851,HM Amla,2,1,JR Hazlewood,caught,0.0,0.0,0.0,5.0,...,K Rabada,SJ Nogajski,RA Kettleborough,AJ Pycroft,NaN,NaN,NaN,South Africa,177.0,NaN
2,1000851,D Elgar,3,1,JR Hazlewood,caught,12.0,2.0,0.0,22.0,...,K Rabada,SJ Nogajski,RA Kettleborough,AJ Pycroft,NaN,NaN,NaN,South Africa,177.0,NaN
3,1000851,JP Duminy,4,1,PM Siddle,caught,11.0,0.0,0.0,23.0,...,K Rabada,SJ Nogajski,RA Kettleborough,AJ Pycroft,NaN,NaN,NaN,South Africa,177.0,NaN
4,1000851,F du Plessis,5,1,MA Starc,caught,37.0,5.0,0.0,74.0,...,K Rabada,SJ Nogajski,RA Kettleborough,AJ Pycroft,NaN,NaN,NaN,South Africa,177.0,NaN


In [7]:
halfcentury = data[data['Bat_Runs']>=50]
halfcentury = halfcentury.filter(['matchid','inning_bat','BattingTeam','Bat_Runs','Players','toss_winner',
'toss_decision'], axis=1)
halfcentury=halfcentury.reset_index(drop = True)
halfcentury['Bat_Runs']=halfcentury.Bat_Runs.astype(int)
doublecentury = data[data['Bat_Runs']>=200]
doublecentury = doublecentury.filter(['matchid','inning_bat','BattingTeam','Bat_Runs','Players','toss_winner',
'toss_decision'], axis=1)
doublecentury=doublecentury.reset_index(drop = True)
doublecentury['Bat_Runs']=doublecentury.Bat_Runs.astype(int)


In [8]:
#inning wise
newdf=pd.DataFrame(columns = ['Match_ID','Home_Team','No_of_50s','Away_Team','No_of_50s','Double_Century'])

In [ ]:
#match id, home team and away team
a = list()
a = data.matchid.unique()
for i in range(len(a)):
    newdf.loc[i,0:1]=a[i]
for i in range(len(df1)):
    for j in range(len(newdf)):
        if(newdf['Match_ID'][j] == df1['matchid'][i]):
            newdf.loc[j,1:2]=df1['toss_winner'][i]
for i in range(len(df1)):
    for j in range(len(newdf)):
        if(newdf['Match_ID'][j] == df1['matchid'][i]):
            if(newdf['Home_Team'][j]!=df1['team1'][i]):
                newdf.loc[j,3:4]=df1['team1'][i]
            else:
                newdf.loc[j,3:4]=df1['team2'][i]
                


In [ ]:
#newfd2 contains data for match-wise
newdf2=newdf.copy()

In [ ]:
#50s scored by home team for inning 1
#this will take 1 hrs 30 min to compute
for i in range(len(newdf)):
    count = 0
    for j in range(len(halfcentury)):
        if(newdf['Match_ID'][i]==halfcentury['matchid'][j] and halfcentury['toss_winner'][j]==newdf['Home_Team'][i] and halfcentury['toss_decision'][j]=='bat'):
            if(halfcentury['inning_bat'][j]==1):        
                count=count+1;
                newdf.loc[i,2:3]=count
    
        elif(newdf['Match_ID'][i]==halfcentury['matchid'][j] and halfcentury['toss_winner'][j]==newdf['Home_Team'][i] and halfcentury['toss_decision'][j]=='field'):
            if(halfcentury['inning_bat'][j]==2):        
                count=count+1;
                newdf.loc[i,2:3]=count
                
        elif(newdf['Match_ID'][i]==halfcentury['matchid'][j] and halfcentury['toss_winner'][j]!=newdf['Home_Team'][i] and halfcentury['toss_decision'][j]=='bat'):
            if(halfcentury['inning_bat'][j]==2):        
                count=count+1;
                newdf.loc[i,2:3]=count
       
        elif(newdf['Match_ID'][i]==halfcentury['matchid'][j] and halfcentury['toss_winner'][j]!=newdf['Home_Team'][i] and halfcentury['toss_decision'][j]=='field'):
             if(halfcentury['inning_bat'][j]==1):        
                count=count+1;
                newdf.loc[i,2:3]=count

for i in range(len(newdf)):
    count = 0
    for j in range(len(halfcentury)):
        if(newdf['Match_ID'][i]==halfcentury['matchid'][j] and halfcentury['toss_winner'][j]==newdf['Away_Team'][i] and halfcentury['toss_decision'][j]=='bat'):
            if(halfcentury['inning_bat'][j]==1):        
                count=count+1;
                newdf.loc[i,4:5]=count
    
        elif(newdf['Match_ID'][i]==halfcentury['matchid'][j] and halfcentury['toss_winner'][j]==newdf['Away_Team'][i] and halfcentury['toss_decision'][j]=='field'):
            if(halfcentury['inning_bat'][j]==2):        
                count=count+1;
                newdf.loc[i,4:5]=count
                
        elif(newdf['Match_ID'][i]==halfcentury['matchid'][j] and halfcentury['toss_winner'][j]!=newdf['Away_Team'][i] and halfcentury['toss_decision'][j]=='bat'):
            if(halfcentury['inning_bat'][j]==2):        
                count=count+1;
                newdf.loc[i,4:5]=count
       
        elif(newdf['Match_ID'][i]==halfcentury['matchid'][j] and halfcentury['toss_winner'][j]!=newdf['Away_Team'][i] and halfcentury['toss_decision'][j]=='field'):
             if(halfcentury['inning_bat'][j]==1):        
                count=count+1;
                newdf.loc[i,4:5]=count                

In [ ]:
#fullmatch-wise for home team
#this will take python 30 mins to computer
for i in range(len(newdf2)):
    count = 0
    for j in range(len(halfcentury)):
        if(newdf2['Match_ID'][i]==halfcentury['matchid'][j] and halfcentury['BattingTeam'][j]==newdf2['Home_Team'][i]):
            count=count+1;
            newdf2.loc[i,2:3]=count

In [ ]:
#fullmatch-wise for away team
#this will take python 30 mins to computer
for i in range(len(newdf2)):
    count = 0
    for j in range(len(halfcentury)):
        if(newdf2['Match_ID'][i]==halfcentury['matchid'][j] and halfcentury['BattingTeam'][j]==newdf2['Away_Team'][i]):
            count=count+1;
            newdf2.loc[i,4:5]=count

In [ ]:
#double century
for i in range(len(newdf2)):
    count = 0
    for j in range(len(doublecentury)):
        if(newdf2['Match_ID'][i]==doublecentury['matchid'][j] and doublecentury['Bat_Runs'][j]>=200):
            count=count+1;
            newdf2.loc[i,5:6]=count

In [ ]:
#double century
for i in range(len(newdf)):
    count = 0
    for j in range(len(doublecentury)):
        if(newdf['Match_ID'][i]==doublecentury['matchid'][j] and doublecentury['Bat_Runs'][j]>=200):
            count=count+1;
            newdf.loc[i,5:6]=count

In [ ]:
#newdf==>inning wise 50s data
#newdf2==>match wise
#display(HTML(newdf.to_html()))
#display(HTML(newdf2.to_html()))

In [ ]:
newdf2.fillna(0)
newdf2.to_csv (r'C:\Users\deepc\Desktop\Data Work\hypothesis_data_MW.csv', index = None, header=True)

In [ ]:
#newdf2 = newdf2.fillna(0)


In [ ]:
newdf.fillna(0)
newdf.to_csv (r'C:\Users\deepc\Desktop\Data Work\hypothesis_data_IW.csv', index = None, header=True)

# HYPOTHESIS DATA

# 1. Half-Century Scored – Home Team Inning 1
# 2. Half Century Scored – Away  Team Inning 1
# 3. Double Century Scored


In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import IPython
from IPython.display import display, HTML
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency

In [51]:
newdf = pd.read_csv(r'C:\Users\deepc\Desktop\Data Work\hypothesis_data_IW.csv')

In [52]:
newdf["No_of_50s_HT"]=newdf.No_of_50s_HT.astype(int)
newdf["No_of_50s_AT"]=newdf.No_of_50s_AT.astype(int)
newdf["Double_Century_AT"]=newdf.Double_Century_AT.astype(int)
newdf["Double_Century_HT"]=newdf.Double_Century_HT.astype(int)

In [55]:
newdf

,Match_ID,Home_Team,No_of_50s_HT,Double_Century_HT,Away_Team,No_of_50s_AT,Double_Century_AT
0,1000851,South Africa,2,0,Australia,2,0
1,1000853,South Africa,2,0,Australia,0,0
2,1000855,South Africa,1,0,Australia,4,0
3,1000881,Australia,3,0,Pakistan,1,0
4,1000883,Pakistan,3,1,Australia,5,0
...,...,...,...,...,...,...,...
406,952193,Papua New Guinea,1,0,Ireland,1,0
407,952195,Namibia,0,0,Afghanistan,1,0
408,995451,Sri Lanka,0,0,Australia,0,0
409,995453,Sri Lanka,2,0,Australia,0,0


In [54]:
#display(HTML(newdf.to_html()))
#newdf.head(20)
contingency_table=pd.crosstab(newdf["Home_Team"],newdf["No_of_50s_HT"])
contingency_table

No_of_50s_HT,0,1,2,3,4,5,6
Home_Team,,,,,,,
Australia,3,8,15,19,7,8,2
Bangladesh,6,7,5,6,1,0,0
England,6,6,19,14,10,3,0
Hong Kong,0,0,0,1,0,0,0
India,4,8,9,12,8,3,1
Ireland,1,1,0,0,0,0,0
Namibia,1,0,0,0,0,0,0
Netherlands,0,0,1,0,0,0,0
New Zealand,6,12,7,8,3,4,0


In [34]:
#odds for halfcentury when comparing with away team
# newdf['total50S']=newdf['No_of_50s_HT']+newdf['No_of_50s_AT']
# newdf['total200s']=newdf['Double_Century_HT']+newdf['Double_Century_AT']
# newdf['oddRatio_50s']=((newdf['No_of_50s_HT']/newdf['total50S'])/(newdf['No_of_50s_AT']/newdf['total50S']))
# newdf['oddRatio_200s']=((newdf['Double_Century_HT']/newdf['total200s'])/(newdf['Double_Century_AT']/newdf['total200s']))
# display(HTML(newdf.fillna(0).to_html()))
# ttest_ind(newdf['No_of_50s_HT'],newdf['No_of_50s_AT'],nan_policy='omit')
# ttest_ind(newdf['Double_Century_HT'],newdf['Double_Century_AT'],nan_policy='omit')

In [35]:
tophc = newdf['No_of_50s_HT'].max()
tophc

6

In [36]:
teams = list()
teams = newdf['Home_Team']
teams = teams.append(newdf['Away_Team'])
teams = teams.unique()
teams

array(['South Africa', 'Australia', 'Pakistan', 'Papua New Guinea',
       'New Zealand', 'India', 'West Indies', 'Zimbabwe', 'England',
       'Bangladesh', 'Sri Lanka', 'Netherlands', 'Ireland', 'Hong Kong',
       'Namibia', 'Afghanistan', 'United Arab Emirates'], dtype=object)

In [37]:
ptest = pd.DataFrame(columns = ['South Africa', 'Australia', 'Pakistan', 'Papua New Guinea',
       'New Zealand', 'India', 'West Indies', 'Zimbabwe', 'England',
       'Bangladesh', 'Sri Lanka', 'Netherlands', 'Ireland', 'Hong Kong',
       'Namibia', 'Afghanistan', 'United Arab Emirates'])

In [38]:
ptest

,South Africa,Australia,Pakistan,Papua New Guinea,New Zealand,India,West Indies,Zimbabwe,England,Bangladesh,Sri Lanka,Netherlands,Ireland,Hong Kong,Namibia,Afghanistan,United Arab Emirates


In [39]:
for i in range(len(teams)):
    zero=0;
    one=0;
#     two=0;
#     three=0;
#     four=0;
#     five=0;
#     six=0;
    for j in range(len(newdf)):
        if(teams[i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==0):
            zero+=1
        if(teams[i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==1):
            one+=1
        if(teams[i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==2):
            one+=1
        if(teams[i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==3):
            one+=1
        if(teams[i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==4):
            one+=1
        if(teams[i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==5):
            one+=1
        if(teams[i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==6):
            one+=1
        if(teams[i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==0):
            zero+=1
        if(teams[i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==1):
            one+=1
        if(teams[i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==2):
            one+=1
        if(teams[i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==3):
            one+=1
        if(teams[i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==4):
            one+=1
        if(teams[i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==5):
            one+=1
        if(teams[i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==6):
            one+=1
    ptest.loc[0,i:i+1]=zero
#     ptest.loc[2,i:i+1]=two
#     ptest.loc[3,i:i+1]=three
#     ptest.loc[4,i:i+1]=four
#     ptest.loc[5,i:i+1]=five
#     ptest.loc[6,i:i+1]=six
    ptest.loc[1,i:i+1]=one

ptest

,South Africa,Australia,Pakistan,Papua New Guinea,New Zealand,India,West Indies,Zimbabwe,England,Bangladesh,Sri Lanka,Netherlands,Ireland,Hong Kong,Namibia,Afghanistan,United Arab Emirates
0,8,9,12,0,13,8,12,3,13,8,11,0,1,0,1,0,0
1,79,104,63,2,71,92,68,15,108,41,71,1,2,2,1,2,1


In [40]:
del ptest['Papua New Guinea']
del ptest['Netherlands']
del ptest['Hong Kong']
del ptest['Afghanistan']

# I X J Table Chi Squared Test

In [59]:
data_frame = ptest
data_frame

,South Africa,Australia,Pakistan,New Zealand,India,West Indies,Zimbabwe,England,Bangladesh,Sri Lanka,Ireland,Namibia,United Arab Emirates
0,8,9,12,13,8,12,3,13,8,11,1,1,0
1,79,104,63,71,92,68,15,108,41,71,2,1,1


In [60]:
iteration_val=len(data_frame.columns)-1
selecte_feature_index=[]

# CHI SQUARED TEST

In [61]:
for i in range(0,iteration_val,1):
    print('Feature Name : {0}'.format(data_frame.columns[i]))
    table= data_frame[data_frame.iloc[:,[i,iteration_val]].columns].values
    chi_squared_stat, p_value, dof, expected=chi2_contingency(table)
    print(chi_squared_stat)
    probability=0.95
    critical=chi2.ppf(probability, dof)
    if abs(chi_squared_stat)>=critical:
        print('Reject Hypothesis 0 (null Hypothesis)')
    elif abs(chi_squared_stat)<critical:
        print('Fail to Reject Hypothesis 0 (null Hypothesis)')
        selecte_feature_index.append(i)
    
    alpha = 1.0 - probability
    print('Significance {0}, {1}'.format(alpha, p_value))
    if p_value<=alpha:
        print("Reject Hypothesis 0 (null Hypothesis)")
    else:
        print('Fail to Reject Hypothesis 0 (null Hypothesis)')
        selecte_feature_index.append(i)
    print('================================================================')

Feature Name : South Africa
2.048275862068965
Fail to Reject Hypothesis 0 (null Hypothesis)
Significance 0.050000000000000044, 0.1523786670550085
Fail to Reject Hypothesis 0 (null Hypothesis)
Feature Name : Australia
2.4596713021491783
Fail to Reject Hypothesis 0 (null Hypothesis)
Significance 0.050000000000000044, 0.11680322244932555
Fail to Reject Hypothesis 0 (null Hypothesis)
Feature Name : Pakistan
0.8919444444444447
Fail to Reject Hypothesis 0 (null Hypothesis)
Significance 0.050000000000000044, 0.3449509253549905
Fail to Reject Hypothesis 0 (null Hypothesis)
Feature Name : New Zealand
0.9408227767602766
Fail to Reject Hypothesis 0 (null Hypothesis)
Significance 0.050000000000000044, 0.33206633432628213
Fail to Reject Hypothesis 0 (null Hypothesis)
Feature Name : India
2.4520329301075265
Fail to Reject Hypothesis 0 (null Hypothesis)
Significance 0.050000000000000044, 0.11737276852896311
Fail to Reject Hypothesis 0 (null Hypothesis)
Feature Name : West Indies
0.9932404891304348
Fa

# ⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️⬆️

# CONTINGENCY OF CHI SQUARED TEST

In [44]:
chi2_contingency(ptest)

(12.278698082994788,
 0.42356464742102257,
 12,
 array([[ 10.56809816,  13.72638037,   9.11042945,  10.20368098,
          12.14723926,   9.71779141,   2.18650307,  14.69815951,
           5.95214724,   9.9607362 ,   0.36441718,   0.24294479,
           0.12147239],
        [ 76.43190184,  99.27361963,  65.88957055,  73.79631902,
          87.85276074,  70.28220859,  15.81349693, 106.30184049,
          43.04785276,  72.0392638 ,   2.63558282,   1.75705521,
           0.87852761]]))

In [45]:
chidata = pd.DataFrame(columns=['team','zero','one','two','three','four','five','six'])

In [46]:
for i in range(len(teams)):
    chidata.loc[i,0:1]=teams[i]


In [47]:
for i in range(len(chidata)):
    zero = 0
    one=0
    two=0
    three=0
    four=0
    five=0
    six=0
    for j in range(len(newdf)):
        if(chidata['team'][i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==0 ):
            zero+=1
        if(chidata['team'][i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==0 ):
            zero+=1
        if(chidata['team'][i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==1 ):
            one+=1
        if(chidata['team'][i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==1 ):
            one+=1
        if(chidata['team'][i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==2 ):
            two+=1
        if(chidata['team'][i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==2 ):
            two+=1
        if(chidata['team'][i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==3 ):
            three+=1
        if(chidata['team'][i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==3 ):
            three+=1
        if(chidata['team'][i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==4 ):
            four+=1
        if(chidata['team'][i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==4 ):
            four+=1
        if(chidata['team'][i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==5 ):
            five+=1
        if(chidata['team'][i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==5 ):
            five+=1
        if(chidata['team'][i]==newdf['Home_Team'][j] and newdf['No_of_50s_HT'][j]==6 ):
            six+=1
        if(chidata['team'][i]==newdf['Away_Team'][j] and newdf['No_of_50s_AT'][j]==6 ):
            six+=1
    chidata.loc[i,1:2]=zero
    chidata.loc[i,2:3]=one
    chidata.loc[i,3:4]=two
    chidata.loc[i,4:5]=three
    chidata.loc[i,5:6]=four
    chidata.loc[i,6:7]=five
    chidata.loc[i,7:8]=six
    
chidata            

,team,zero,one,two,three,four,five,six
0,South Africa,8,15,26,20,13,5,0
1,Australia,9,20,29,26,16,11,2
2,Pakistan,12,12,18,15,15,3,0
3,Papua New Guinea,0,2,0,0,0,0,0
4,New Zealand,13,24,17,13,11,5,1
5,India,8,16,24,26,13,11,2
6,West Indies,12,22,23,14,5,3,1
7,Zimbabwe,3,5,4,5,1,0,0
8,England,13,15,38,27,22,6,0
9,Bangladesh,8,15,8,13,3,2,0


In [49]:
chidata.to_csv (r'C:\Users\deepc\Desktop\Data Work\hypo.csv', index = None, header=True)